In [2]:
# ============================================================
# CNN Advanced Practice Notebook 01
# Baseline CNN vs Data Augmentation (CIFAR-10)
# ============================================================
# [KR]
# 목표:
# - 기본 CNN(Baseline)과 데이터 증강(Data Augmentation)을 적용한 CNN의
#   분류 성능을 비교한다.
#
# [JP]
# 目的:
# - ベースラインCNNとデータ拡張(Data Augmentation)を適用したCNNの
#   分類性能を比較する。
#
# Dataset: CIFAR-10
# - 32×32 RGB 컬러 이미지
# - 10개 클래스 분류용 표준 벤치마크 데이터셋

# Architecture:
# - Conv2D (32 filters) + MaxPooling
# - Conv2D (64 filters) + MaxPooling
# - Fully Connected Layer (64 units)
# - Output Layer (Softmax, 10 classes)

In [7]:
# ------------------------------------------------------------
# 0) Library Import
# ------------------------------------------------------------
import tensorflow as tf

# Sequential: 레이어를 순서대로 쌓는 가장 기본적인 모델 방식
from tensorflow.keras.models import Sequential

# CNN 주요 레이어들
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 데이터 증강(Data Augmentation)을 쉽게 적용하기 위한 도구
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [8]:
# ------------------------------------------------------------
# 1) Data Load & Preprocessing
# ------------------------------------------------------------
# CIFAR-10 데이터 로드
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# 픽셀값 정규화
# 0~255 범위의 정수값을 0~1 사이의 실수값으로 변환
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# Train / Validation 데이터 분리
# - stratify=y_train: 클래스 비율 유지
# - random_state 고정: 실험 재현성 확보
x_tr, x_val, y_tr, y_val = train_test_split(
    x_train, y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

In [9]:
# ------------------------------------------------------------
# 2) Baseline CNN Model Definition
# ------------------------------------------------------------
# Conv -> Pool -> Conv -> Pool -> Flatten -> Dense -> Output
# 비교 실험을 위해 구조는 단순하게 유지
def build_model(input_shape=(32, 32, 3), num_classes=10):

    model = Sequential([
        # 첫 번째 합성곱 층
        # - 기본적인 엣지/색상 특징 추출
        Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
        MaxPooling2D(),

        # 두 번째 합성곱 층
        # - 더 복잡한 패턴/텍스처 특징 학습
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D(),

        # 2D feature map을 1D 벡터로 변환
        Flatten(),

        # Fully Connected Layer
        Dense(64, activation="relu"),

        # Output Layer (10개 클래스)
        Dense(num_classes, activation="softmax"),
    ])

    # 컴파일
    # - sparse_categorical_crossentropy:
    #   one-hot 인코딩이 아닌 정수 라벨을 그대로 사용
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [10]:
model = build_model()
model.summary()

C:\Users\JA60301\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2304)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         147,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 167,562 (654.54 KB)

 Trainable params: 167,562 (654.54 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# ------------------------------------------------------------
# 3) Data Augmentation 설정
# ------------------------------------------------------------
# 학습 데이터에 다양한 변형을 주어
# - 과적합(Overfitting) 감소
# - 일반화 성능 향상
datagen = ImageDataGenerator(
    rotation_range=20,        # 이미지 회전
    width_shift_range=0.1,    # 가로 방향 이동
    height_shift_range=0.1,   # 세로 방향 이동
    horizontal_flip=True      # 좌우 반전
)

# ※ Validation/Test 데이터에는 증강을 적용하지 않음

In [6]:
# ------------------------------------------------------------
# 4) Model Training: Baseline (No Augmentation)
# ------------------------------------------------------------
EPOCHS = 20
BATCH_SIZE = 64

model_base = build_model()

history_base = model_base.fit(
    x_tr, y_tr,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(x_val, y_val),
    verbose=1
)

C:\Users\JA60301\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.4394 - loss: 1.5626 - val_accuracy: 0.5346 - val_loss: 1.3016
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.5735 - loss: 1.2137 - val_accuracy: 0.6092 - val_loss: 1.1316
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.6313 - loss: 1.0590 - val_accuracy: 0.6455 - val_loss: 1.0353
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.6644 - loss: 0.9675 - val_accuracy: 0.6514 - val_loss: 1.0077
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.6860 - loss: 0.8986 - val_accuracy: 0.6660 - val_loss: 0.9690
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.7095 - loss: 0.8418 - val_accuracy: 0.6549 - val_loss: 0.9958
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.7235 - loss: 0.7948 - val_accuracy: 0.6726 - val_loss: 0.9468
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.7390 - loss: 0.7518 - val_a

In [7]:
# ------------------------------------------------------------
# 5) Model Training: Augmented
# ------------------------------------------------------------
# datagen.flow():
# - 매 epoch마다 랜덤하게 변형된 이미지를 생성하여 학습
model_aug = build_model()

history_aug = model_aug.fit(
    datagen.flow(x_tr, y_tr, batch_size=BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=(x_val, y_val),
    verbose=1
)

Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 44ms/step - accuracy: 0.3787 - loss: 1.7107 - val_accuracy: 0.5079 - val_loss: 1.3746
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.4870 - loss: 1.4302 - val_accuracy: 0.5522 - val_loss: 1.2577
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.5267 - loss: 1.3250 - val_accuracy: 0.5949 - val_loss: 1.1632
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.5585 - loss: 1.2434 - val_accuracy: 0.6184 - val_loss: 1.0968
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.5774 - loss: 1.1944 - val_accuracy: 0.6382 - val_loss: 1.0512
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.5901 - loss: 1.1607 - val_accuracy: 0.6451 - val_loss: 1.0314
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 44ms/step - accuracy: 0.5999 - loss: 1.1272 - val_accuracy: 0.6480 - val_loss: 1.0162
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 44ms/step - accuracy: 0.6147 - loss: 1.0966 - 

In [1]:
# ------------------------------------------------------------
# 6) Result Visualization
# ------------------------------------------------------------
# Baseline vs Augmentation 검증 정확도 비교
plt.plot(history_base.history["val_accuracy"], label="Baseline")
plt.plot(history_aug.history["val_accuracy"], label="Augmented")

plt.title("Validation Accuracy Comparison (CIFAR-10)")
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.legend()

plt.savefig("notebook01_val_accuracy.png", dpi=300, bbox_inches="tight")

plt.show()

# - Baseline 모델은 초기 epoch에서 빠르게 성능이 향상되지만,
#   이후 validation accuracy가 정체되는 경향을 보인다.
# - Data Augmentation을 적용한 모델은 epoch이 증가함에 따라
#   validation accuracy가 보다 안정적으로 향상된다.
# - 이는 데이터 증강이 입력 데이터의 다양성을 증가시켜
#   과적합을 완화하고 일반화 성능을 향상시킴을 의미한다.

# - ベースラインモデルは初期のepochで急激に性能が向上するが、
#   その後 validation accuracy が停滞する傾向が見られる。
# - データ拡張を適用したモデルは、epochの増加とともに
#   validation accuracyがより安定して向上する。
# - これは、データ拡張によって入力データの多様性が増加し、
#   過学習が抑制され、汎化性能が向上したことを示している。

NameError: name 'plt' is not defined